In [72]:
import pandas
import requests #use requests over 'urllib.request'. Work-around for encoding problem. See: https://stackoverflow.com/questions/47419104/unicodedecodeerror-utf-8-codec-cant-decode-python3
import json
import geojson
import folium

In [90]:
#GDELT GeoJson Endpoint
url = "https://api.gdeltproject.org/api/v1/gkg_geojson?MAXROWS=50"

#Issue request to URL
request = requests.get(url)
print(request.status_code)
if request.status_code == 200: data = request.json()
print(type(data))
# print("---------")
# print(data['features'])

200
<class 'dict'>


In [91]:
#I think this solution works!
#However, this will probably just work for my data only.

flat_dictionary = {}
for feature in data['features']:
    for key, value in feature.items():
      if type(feature[key]) == dict:
        for k, v in feature[key].items():
            if k in flat_dictionary.keys():
                flat_dictionary[k].append(feature[key][k])
            else:
                flat_dictionary[k] = [v]
   
df_features = pandas.DataFrame.from_dict(flat_dictionary)

#from: https://stackoverflow.com/questions/35491274/pandas-split-column-of-lists-into-multiple-columns
df_features[['lon','lat']] = pandas.DataFrame(df_features.coordinates.values.tolist(),index = df_features.index)
df_features

,coordinates,mentionedthemes,name,type,url,urlpubtimedate,urltone,lon,lat
0,"[77.5833, 12.9833]",,"Bangalore, Karnataka, India",Point,http://www.4-traders.com/APPLE-4849/news/Apple...,2018-02-02T00:00:00Z,0.65,77.5833,12.9833
1,"[105.0, 35.0]",;GEN_HOLIDAY;,China,Point,http://www.4-traders.com/APPLE-4849/news/Apple...,2018-02-02T00:00:00Z,0.65,105.0000,35.0000
2,"[-122.419, 37.7749]",,"San Francisco, California, United States",Point,http://www.4-traders.com/APPLE-4849/news/Apple...,2018-02-02T00:00:00Z,0.65,-122.4190,37.7749
3,"[2.0, 46.0]",;CRISISLEX_CRISISLEXREC;SECURITY_SERVICES;WB_6...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000
4,"[2.0, 46.0]",;TRIAL;TAX_FNCACT_PROSECUTORS;TAX_FNCACT_WOMEN...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000
5,"[32.0, 49.0]",;TAX_FNCACT_CHILD;SCANDAL;SCANDAL;TAX_ETHNICIT...,Ukraine,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,32.0000,49.0000
6,"[2.0, 46.0]",;SCANDAL;SCANDAL;TAX_ETHNICITY_UKRAINIAN;TAX_W...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000
7,"[2.0, 46.0]",;SCANDAL;SCANDAL;TAX_ETHNICITY_UKRAINIAN;TAX_W...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000
8,"[2.0, 46.0]",;TAX_WEAPONS_EXPLOSIVES;SCANDAL;TAX_FOODSTAPLE...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000
9,"[2.0, 46.0]",;SCANDAL;TAX_FOODSTAPLES_MILK;WB_1458_HEALTH_P...,France,Point,http://infoline.ua/main/world/Skandal-s-francu...,2018-02-02T00:00:00Z,-4.14,2.0000,46.0000


In [92]:
#map with folium
mean_lat = df_features.mean().lat
mean_lon = df_features.mean().lon

m = folium.Map(location=[mean_lat,mean_lon],zoom_start=2.5)
feature_group = folium.FeatureGroup(name='events')

for lat,lon in zip(df_features.lat,df_features.lon):
    folium.CircleMarker(
        radius=5,
        location = [lat,lon],
        popup="event",
        color='crimson',
        fill=False,
        ).add_to(feature_group)
    
feature_group.add_to(m)
m

In [68]:
#another thing to try
#via https://www.haykranen.nl/2016/02/13/handling-complex-nested-dicts-in-python/

# class DictQuery(dict):
#     def get(self, path, default = None):
#         keys = path.split("/")
#         val = None

#         for key in keys:
#             if val:
#                 if isinstance(val, list):
#                     val = [ v.get(key, default) if v else None for v in val]
#                 else:
#                     val = val.get(key, default)
#             else:
#                 val = dict.get(self, key, default)

#             if not val:
#                 break;

#         return val

In [69]:
# from: https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json    
#a function that flattens dictionaries into lists
#I'm not sure how to implement this for my data...
# def dict_generator(indict, pre=None):
#     pre = pre[:] if pre else []
#     if isinstance(indict, dict):
#         for key, value in indict.items():
#             if isinstance(value, dict):
#                 for d in dict_generator(value, [key] + pre):
#                     yield d
#             elif isinstance(value, list) or isinstance(value, tuple):
#                 for v in value:
#                     for d in dict_generator(v, [key] + pre):
#                         yield d
#             else:
#                 yield pre + [key, value]
#     else:
#         yield indict
        
# for i in dict_generator(data):
#     print(i)